In [4]:
#!pip install Levenshtein
#!pip install sentence_transformers

     -------------------------------------- 163.3/163.3 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 388.9/388.9 kB 1.2 MB/s eta 0:00:00
     ---------------------------------------- 8.8/8.8 MB 618.4 kB/s eta 0:00:00
     ------------------------------------ 172.0/172.0 kB 865.0 kB/s eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 919.1 kB/s eta 0:00:00
     -------------------------------------- 269.7/269.7 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.7.1
    Uninstalling fsspec-2022.7.1:
      Successfully uninstalled fsspec-2022.7.1
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.13.2
    Uninstalling huggingface-hub-0.13.2:
      Successfully uninstalled huggingface-hub-0.13.2
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13

In [5]:
import intellikit as ik
import pandas as pd

In [18]:
# Define your DataFrame on which the recommendation system is going to be based
df = pd.DataFrame({
    'feature1': ['apple', 'orange', 'banana', 'grape'],
    'feature2': ['red', 'green', 'blue', 'yellow'],
    'feature3': ['small', 'large', 'medium', 'small'],
    'feature4': [1, 2, 3, 4],
    'feature5': [10, 20, 30, 40]
})

#Define your query 
query = pd.DataFrame({
    'feature1': ['apple'],
    'feature2': ['yellow'],
    'feature3': ['small'],
    'feature4': [3],
    'feature5': [30]

})

#Define you similarity calculation methods for your project
hamming = ik.sim_hamming
levenshtein = ik.sim_levenshtein
ngrams = ik.sim_ngram
abs_diff = ik.sim_difference

# Assign the approriate similarity calculation functions to each feature
similarity_functions = {
    'feature1': hamming,
    'feature2': levenshtein,
    'feature3': ngrams,
    'feature4': abs_diff,
    'feature5': abs_diff
}

In [19]:
unweighted_results = [similarity_functions[feature](df, query, feature) for feature in df.columns]

# Concatenate the resulting DataFrame columns along the rows
result = pd.concat(unweighted_results, axis=1)

print(result)
 

   feature1  feature2  feature3  feature4  feature5
0         0         5       1.0  0.333333  0.333333
1         5         6       0.0  0.666667  0.666667
2         5         5       0.0  1.000000  1.000000
3         4         0       1.0  0.750000  0.750000


In [20]:
#Applying the methods and weights and retreiving the top results
# Define weights for each feature
weights = {
    'feature1': 0.25,
    'feature2': 0.3,
    'feature3': 0.25,
    'feature4': 0.1,
    'feature5': 0.1
}

# Multiply each column by its corresponding weight
weighted_df = result * pd.Series(weights)

# Add a new column for the total of the weighted columns
weighted_df['weighted_total'] = weighted_df.sum(axis=1)

print(weighted_df)

   feature1  feature2  feature3  feature4  feature5  weighted_total
0      0.00       1.5      0.25  0.033333  0.033333        1.816667
1      1.25       1.8      0.00  0.066667  0.066667        3.183333
2      1.25       1.5      0.00  0.100000  0.100000        2.950000
3      1.00       0.0      0.25  0.075000  0.075000        1.400000


In [21]:
#You then go ahead and sort by the weighted total and retreived the top k ranked cases